In [10]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3
import pandas as pd
from PIL import Image

import numpy as np
from matplotlib import pyplot as plt
import time

In [5]:
matlab_csv_s3_url = 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/biomass.matlab.imr.pen-61-from-2019-09-13-to-2019-09-23.results.csv'
s3_access_utils = S3AccessUtils('/root/data')
matlab_csv_f, bucket, key = s3_access_utils.download_from_url(matlab_csv_s3_url)
matlab_df = pd.read_csv(matlab_csv_f)


In [9]:
image_f, _, _ = s3_access_utils.download_from_url(matlab_df.left_crop_url.iloc[0])

In [15]:
np.array(Image.open(image_f)).shape

(662, 2600, 3)

In [17]:
matlab_df

,group_id,site_id,pen_id,left_crop_url,right_crop_url,annotation,akpd_score,estimated_weight_g,estimated_length_mm,captured_at,quality_score
0,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 0, ""leftCrop"": [{""xCrop"": 471, ""yCrop"":...",0.000000e+00,2326.844037,504.349335,2019-09-13T00:00:57.613770000Z,NaN
1,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 1, ""leftCrop"": [{""xCrop"": 118, ""yCrop"":...",0.000000e+00,-294.071492,3.951101,2019-09-13T00:07:27.701929000Z,NaN
2,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 2, ""leftCrop"": [{""xCrop"": 1531, ""yCrop""...",5.000000e-10,2239.708751,462.771149,2019-09-13T00:09:55.074253000Z,NaN
3,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 3, ""leftCrop"": [{""xCrop"": 49, ""yCrop"": ...",9.946678e-01,2553.666234,652.931213,2019-09-13T00:14:06.922994000Z,NaN
4,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 4, ""leftCrop"": [{""xCrop"": 140, ""yCrop"":...",0.000000e+00,2813.315392,549.319458,2019-09-13T00:14:06.922994000Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11967,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 11920, ""leftCrop"": [{""xCrop"": 1257, ""yC...",0.000000e+00,3561.717272,337.898041,2019-09-21T18:33:42.339153000Z,NaN
11968,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 11921, ""leftCrop"": [{""xCrop"": 89, ""yCro...",9.790332e-01,3497.367501,664.067261,2019-09-21T18:37:41.217342000Z,NaN
11969,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 11922, ""leftCrop"": [{""xCrop"": 102, ""yCr...",0.000000e+00,1428.003311,374.888336,2019-09-21T18:40:21.933114000Z,NaN
11970,61,40,61,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,https://s3-eu-west-1.amazonaws.com/aquabyte-cr...,"{""id"": 11923, ""leftCrop"": [{""xCrop"": 1608, ""yC...",8.927323e-01,2723.460793,637.815186,2019-09-21T18:42:57.105974000Z,NaN
